In [1]:
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.layers import Input
import tensorflow as tf
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense


In [ ]:
#model = keras.models.Sequential()
#model.add(VGG16(weights='imagenet'))
#model.add(ResNet50(weights='imagenet'))
#model.summary()

In [6]:
#inp = Input(shape=(224,224,3))
#out = ResNet50(weights='imagenet')(inp)
#classifier = tf.keras.Model(inputs=inp, outputs=out)
#out = VGG16(weights='imagenet')(inp)
#model = tf.keras.Model(inputs=inp, outputs=out)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present
TrainingImagePath='/content/drive/MyDrive/Colab Notebooks/ibb_ass3/train'

from keras.preprocessing.image import ImageDataGenerator
# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 746 images belonging to 100 classes.
Found 746 images belonging to 100 classes.


{'001': 0,
 '002': 1,
 '003': 2,
 '004': 3,
 '005': 4,
 '006': 5,
 '007': 6,
 '008': 7,
 '009': 8,
 '010': 9,
 '011': 10,
 '012': 11,
 '013': 12,
 '014': 13,
 '015': 14,
 '016': 15,
 '017': 16,
 '018': 17,
 '019': 18,
 '020': 19,
 '021': 20,
 '022': 21,
 '023': 22,
 '024': 23,
 '025': 24,
 '026': 25,
 '027': 26,
 '028': 27,
 '029': 28,
 '030': 29,
 '031': 30,
 '032': 31,
 '033': 32,
 '034': 33,
 '035': 34,
 '036': 35,
 '037': 36,
 '038': 37,
 '039': 38,
 '040': 39,
 '041': 40,
 '042': 41,
 '043': 42,
 '044': 43,
 '045': 44,
 '046': 45,
 '047': 46,
 '048': 47,
 '049': 48,
 '050': 49,
 '051': 50,
 '052': 51,
 '053': 52,
 '054': 53,
 '055': 54,
 '056': 55,
 '057': 56,
 '058': 57,
 '059': 58,
 '060': 59,
 '061': 60,
 '062': 61,
 '063': 62,
 '064': 63,
 '065': 64,
 '066': 65,
 '067': 66,
 '068': 67,
 '069': 68,
 '070': 69,
 '071': 70,
 '072': 71,
 '073': 72,
 '074': 73,
 '075': 74,
 '076': 75,
 '077': 76,
 '078': 77,
 '079': 78,
 '080': 79,
 '081': 80,
 '082': 81,
 '083': 82,
 '084': 83,
 '

In [4]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: '001', 1: '002', 2: '003', 3: '004', 4: '005', 5: '006', 6: '007', 7: '008', 8: '009', 9: '010', 10: '011', 11: '012', 12: '013', 13: '014', 14: '015', 15: '016', 16: '017', 17: '018', 18: '019', 19: '020', 20: '021', 21: '022', 22: '023', 23: '024', 24: '025', 25: '026', 26: '027', 27: '028', 28: '029', 29: '030', 30: '031', 31: '032', 32: '033', 33: '034', 34: '035', 35: '036', 36: '037', 37: '038', 38: '039', 39: '040', 40: '041', 41: '042', 42: '043', 43: '044', 44: '045', 45: '046', 46: '047', 47: '048', 48: '049', 49: '050', 50: '051', 51: '052', 52: '053', 53: '054', 54: '055', 55: '056', 56: '057', 57: '058', 58: '059', 59: '060', 60: '061', 61: '062', 62: '063', 63: '064', 64: '065', 65: '066', 66: '067', 67: '068', 68: '069', 69: '070', 70: '071', 71: '072', 72: '073', 73: '074', 74: '075', 75: '076', 76: '077', 77: '078', 78: '079', 79: '080', 80: '081', 81: '082', 82: '083', 83: '084', 84: '085', 85: '086', 86: '087', 87: '088', 88: '089', 89:

In [5]:
model = keras.models.Sequential()
#model.add(VGG16(include_top=False, input_shape=(224,224,3)))
model.add(ResNet50(include_top=False, input_shape=(224,224,3)))

94781440/94765736 [==============================] - 0s 0us/step


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 512)         26214912  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 512)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 100)               25700     
                                                        

In [7]:
#dountttttttt

model.add(Convolution2D(512, kernel_size=(5, 5), strides=(1, 1), input_shape=(224,224,3), activation='relu'))
#model.add(Convolution2D(32, kernel_size=3, strides=(1, 1)))
model.add(MaxPool2D(pool_size=(2,2)))




In [8]:
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(OutputNeurons, activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])


In [11]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit_generator(
                    training_set,
                    steps_per_epoch=22,
                    epochs=20,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/20
22/22 [==============================] - 578s 26s/step - loss: 5.6273 - accuracy: 0.0073 - val_loss: 23213.8223 - val_accuracy: 0.0156
Epoch 2/20
22/22 [==============================] - 560s 25s/step - loss: 4.6112 - accuracy: 0.0205 - val_loss: 2512.0168 - val_accuracy: 0.0063
Epoch 3/20
22/22 [==============================] - 550s 25s/step - loss: 4.6010 - accuracy: 0.0103 - val_loss: 65.2785 - val_accuracy: 0.0156
Epoch 4/20
22/22 [==============================] - 551s 25s/step - loss: 4.5797 - accuracy: 0.0176 - val_loss: 7.1909 - val_accuracy: 0.0094
Epoch 5/20
22/22 [==============================] - 553s 25s/step - loss: 4.5284 - accuracy: 0.0220 - val_loss: 6.6529 - val_accuracy: 0.0094
Epoch 6/20
22/22 [==============================] - 551s 25s/step - loss: 4.5103 - accuracy: 0.0220 - val_loss: 4.5592 - val_accuracy: 0.0344
Epoch 7/20
22/22 [==============================] - 549s 25s/step - loss: 4.4574 - accuracy: 0.0323 - val_loss: 4.5268 - val_accuracy: 0.046

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/ibb_ass3")